In [1]:
import pandas as pd
import os
import cx_Oracle
pd.set_option('display.max_columns', 77)
os.environ["NLS_LANG"] = ".UTF8"
import numpy as np
import matplotlib
%matplotlib inline 
#connect.version
pd.options.display.max_rows=300

In [2]:
import sys
import csv
#write data into a text file
#conn = cx_Oracle.connect('login/password@db.local/bird')

#sql = "select hdrid,posdat,tgross,clntno from table where posdat BEtween TO_DATE('2017-06-01','YYYY-MM-DD') and TO_DATE('2017-06-30','YYYY-MM-DD') and CLNTNO not in ('5550012345678','9991000445568','0101000000004','0100000020005') order by posdat"
#sql= "Select CLNTNO,count(*) as frequency,(TO_DATE('2017-06-30','YYYY-MM-DD') - max(TRUNC(posdat))) as recency, SUM(TGROSS) as monetary_value FROM table where posdat BETWEEN TO_DATE('2013-01-01','YYYY-MM-DD') AND TO_DATE('2017-06-30','YYYY-MM-DD') AND CLNTNO not in ('5550012345678','9991000445568','0101000000004','0100000020005') group by CLNTNO"
sql="SELECT MIN(TO_CHAR(posdat,'YYYY')) || MIN(TO_CHAR(posdat,'Q')) as minDate,CLNTNO,(TO_DATE('2017-06-30','YYYY-MM-DD') - MAX(TRUNC(posdat))) as Recency,SUM(TGROSS) as Monetary_value,COUNT(*) as Frequency \
FROM db.table \
WHERE CLNTNO NOT in ('5550012345678','9991000445568','0101000000004','0100000020005') \
AND posdat <= TO_DATE('2017-06-30','YYYY-MM-DD') \
GROUP BY CLNTNO \
HAVING MIN(TO_CHAR(posdat,'YYYY-MM-DD')) > '2013-01-01'"
curs2 = conn.cursor()
curs2.execute(sql)
printHeader = True 

for row_data in curs2:
    #if not row_data[0].startswith('BIN$'): 
        tableName = row_data[0]

        csv_file_dest = "rfmlData.csv"
        outputFile = open(csv_file_dest,'w') # 'wb'
        output = csv.writer(outputFile, dialect='excel')    
        

        if printHeader:
            cols = []
            for col in curs2.description:
                cols.append(col[0])
            output.writerow(cols)

        for row_data in curs2: 
            output.writerow(row_data)

        outputFile.close()

In [3]:
!grep -ve '\"' -ve "Ф" -ve '!' rfmlData.csv | awk -F',' '$4>0' > rfmlData_woTrash.csv #drop all trash from data
#!cut -d ',' -f2 rfmlData_woTrash.csv | sort -u | wc -l && wc -l rfmlData_woTrash.csv 

In [4]:
dfile2 = pd.read_csv('rfmlData_woTrash.csv',delimiter=",",engine="c",memory_map=True)

/usr/local/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
dfile2.sort_values(by='MONETARY_VALUE',kind='mergesort',ascending=False)
#dfile2.loc[dfile2['MONETARY_VALUE']<=0]

,MINDATE,CLNTNO,RECENCY,MONETARY_VALUE,FREQUENCY
1044186,20171,99904273441,6,1.735300e+08,753
1661672,20161,99905040218,144,1.415345e+08,2080
1660648,20161,99904947137,0,9.103455e+07,1334
4840070,20161,99904852919,9,9.013654e+07,413
4844412,20162,99905416291,217,7.436293e+07,1527
3592523,20161,99904875644,1,7.261420e+07,2762
2037989,20171,99905889054,130,7.234118e+07,868
1985919,20171,99905881724,81,7.045042e+07,273
5815199,20171,99904950987,53,6.573373e+07,242
2269854,20164,99904867293,219,5.825785e+07,678


In [6]:
quantiles = dfile2.quantile(q=[0.25,0.5,0.75]).to_dict()
rfmlSegment = dfile2
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

quantiles # how looks our quantiles segmentation in value's diapason

{'FREQUENCY': {0.25: 1.0, 0.5: 2.0, 0.75: 6.0},
 'MINDATE': {0.25: 20141.0, 0.5: 20151.0, 0.75: 20161.0},
 'MONETARY_VALUE': {0.25: 1112.0, 0.5: 3065.0, 0.75: 9555.0},
 'RECENCY': {0.25: 113.0, 0.5: 398.0, 0.75: 858.0}}

In [7]:
rfmlSegment['R_Quartile'] = rfmlSegment['RECENCY'].apply(RClass, args=('RECENCY',quantiles,))
rfmlSegment['F_Quartile'] = rfmlSegment['FREQUENCY'].apply(FMClass, args=('FREQUENCY',quantiles,))
rfmlSegment['M_Quartile'] = rfmlSegment['MONETARY_VALUE'].apply(FMClass, args=('MONETARY_VALUE',quantiles,))
rfmlSegment['L_Quartile'] = rfmlSegment['MINDATE'].apply(RClass, args=('MINDATE',quantiles,))


rfmlSegment['RFMLabel'] = rfmlSegment.R_Quartile.map(str) \
                            + rfmlSegment.F_Quartile.map(str) \
                            + rfmlSegment.M_Quartile.map(str) \
                            + rfmlSegment.L_Quartile.map(str)
        #make lables for each items in array
rfmlSegment['RFMLabel'].value_counts() #print quantity of each RFM-segment

1111    350886
3443    302990
4441    287737
4442    238068
2444    237564
2111    212933
4431    168725
1112    166373
3433    158072
1444    147029
2434    135494
4432    127350
3111    113425
1113    101015
4221     88743
4331     85560
3221     82358
1434     81343
2112     76431
2424     76430
3423     71545
4231     68410
2223     68250
1223     64756
1224     63847
3222     60131
3333     59217
4421     58343
2222     57835
4321     54887
1334     53491
4341     53160
2121     51040
2334     50197
3231     48792
2221     48332
4422     47453
3121     45729
3332     42832
3232     41862
1424     41337
1121     41096
4111     41005
1324     40599
4332     39915
2233     39910
3444     39733
2324     38968
3442     38009
3323     37755
1234     37437
2213     37243
4211     36953
1213     36022
1122     35301
3343     33472
2333     33167
3211     32967
2224     32530
1123     32499
1222     31769
3223     31369
1214     31105
2212     30628
1233     29936
2232     29612
2122     2

In [8]:
rfmlSegment #show final table

,MINDATE,CLNTNO,RECENCY,MONETARY_VALUE,FREQUENCY,R_Quartile,F_Quartile,M_Quartile,L_Quartile,RFMLabel
0,20131,99902729797,1042,14526.0,5,4,2,1,1,4211
1,20131,99902505392,809,38989.0,12,3,1,1,1,3111
2,20131,99902677912,78,6616.0,5,1,2,2,1,1221
3,20131,5550150854022,1265,1918.0,3,4,2,3,1,4231
4,20141,5550158674820,114,11136.0,22,2,1,1,1,2111
5,20151,99904073961,22,59472.0,30,1,1,1,2,1112
6,20151,5550171109033,480,3129.0,4,3,2,2,2,3222
7,20131,99902735909,21,38374.0,24,1,1,1,1,1111
8,20151,99903870056,343,599337.0,125,2,1,1,2,2112
9,20141,99903704607,10,228374.0,99,1,1,1,1,1111


In [9]:
rfmlSegment['L_Quartile'].value_counts() #print quantity of each RFM-segment

1    2204296
2    1430064
3    1328041
4    1319707
Name: L_Quartile, dtype: int64

In [11]:
rfmlSegment.to_csv("Segment_RG_clients.csv",index=False)